In [26]:
class PriceBoard:
    def __init__(self, ask0, bid0, info=False):
        self.ask = ask0
        self.bid = bid0
        self.market_ask = 0
        self.limit_ask = []
        self.contracted_ask = []
        self.market_bid = 0
        self.limit_bid = []
        self.contracted_bid = []
        self.info = info
        return
    
    @property
    def spread(self):
        return self.bid - self.ask
    
    def order_ask(self, volume, price=None):
        if price is None:
            # markert order
            self.market_ask += volume
            if self.info:
                print("ORDER ASK: TYPE=market VOLUME=%d" % volume)
        else:
            # limit order
            self.limit_ask.append([volume, price])
            if self.info:
                print("ORDER ASK: TYPE=limit VOLUME=%d PRICE=%g" % (volume, price))
        return
    
    def order_bid(self, volume, price=None):
        if price is None:
            # markert order
            self.market_bid += volume
            if self.info:
                print("ORDER BID: TYPE=market VOLUME=%d" % volume)
        else:
            # limit order
            self.limit_bid.append([volume, price])
            if self.info:
                print("ORDER BID: TYPE=limit VOLUME=%d PRICE=%g" % (volume, price))
        return
    
    def sort_order(self):
        import numpy as np
        
        arg_limit_ask = np.argsort([a[1] for a in self.limit_ask])[::-1]
        self.limit_ask = [self.limit_ask[i] for i in arg_limit_ask]
        arg_limit_bid = np.argsort([b[1] for b in self.limit_bid])
        self.limit_bid = [self.limit_bid[j] for j in arg_limit_bid]
        return
    
    def clear_market_order(self):
        self.market_ask = 0
        self.market_bid = 0
        return
     
    def contract(self):
        import numpy as np
        
        # sort order
        self.sort_order()
        
        # clear contracted order
        self.contracted_ask = []
        self.contracted_bid = []
        
        # contract order
        if self.market_ask > self.market_bid:
            
            # market order
            median_price = np.mean([self.ask, self.bid])
            self.contracted_bid.append((self.market_bid, median_price))
            self.contracted_ask.append((self.market_bid, median_price))
            self.market_ask -= self.market_bid
            self.market_bid = 0
            if self.info:
                print("CONTRACT(A): volume=%d price=%g" % tuple(self.contracted_bid[-1]))
            
            # limit order
            for i, (volume, price) in enumerate(self.limit_bid):
                if volume < self.market_ask:
                    self.contracted_ask.append((volume, price))
#                     self.contracted_bid.append((volume, price))
                    self.market_ask -= volume
                    self.limit_bid[i][0] = 0  # change volume to zero
                    if self.info:
                        print("CONTRACT(A): volume=%d price=%g" % tuple(self.contracted_bid[-1]))
                else:
                    self.contracted_ask.append((self.market_ask, price))
#                     self.contracted_bid.append((self.market_ask, price))
                    self.limit_bid[i][0] -= self.market_ask
                    self.market_ask = 0
                    if self.info:
                        print("CONTRACT(A): volume=%d price=%g" % tuple(self.contracted_bid[-1]))
                    break
                    
            # update limit order
            self.limit_bid = [
                a for a in self.limit_bid
                if a[0] >= 1.
            ]
                    
        elif self.market_ask < self.market_bid:
            
            # market order
            median_price = np.mean([self.ask, self.bid])
            self.contracted_bid.append((self.market_ask, median_price))
            self.contracted_ask.append((self.market_ask, median_price))
            self.market_bid -= self.market_ask
            self.market_ask = 0
            if self.info:
                print("CONTRACT(B): volume=%d price=%g" % tuple(self.contracted_ask[-1]))
            
            # limit order
            for i, (volume, price) in enumerate(self.limit_ask):
                if volume < self.market_bid:
#                     self.contracted_ask.append((volume, price))
                    self.contracted_bid.append((volume, price))
                    self.market_bid -= volume
                    self.limit_ask[i][0] = 0
                    if self.info:
                        print("CONTRACT(B): volume=%d price=%g" % tuple(self.contracted_ask[-1]))
                else:
#                     self.contracted_ask.append((self.market_bid, price))
                    self.contracted_bid.append((self.market_bid, price))
                    self.limit_ask[i][0] -= self.market_bid
                    self.market_bid = 0
                    if self.info:
                        print("CONTRACT(B): volume=%d price=%g" % tuple(self.contracted_ask[-1]))
                    break
            
            # update limit order
            self.limit_ask = [
                a for a in self.limit_ask
                if a[0] >= 1.
            ]
            
        else:
            # market order
            median_price = np.mean([self.ask, self.bid])
            self.contracted_bid.append([self.market_ask, median_price])
            self.contracted_ask.append([self.market_ask, median_price])
            self.market_ask = 0
            self.market_bid = 0
            if self.info:
                print("CONTRACT(C): volume=%d price=%g" % tuple(self.contracted_ask[-1]))
            
        # update price
        self.ask = np.sum(
            [a[0] * a[1] for a in self.contracted_ask]
        ) / np.sum(
            [a[0] for a in self.contracted_ask]
        )
        self.bid = np.sum(
            [a[0] * a[1] for a in self.contracted_bid]
        ) / np.sum(
            [a[0] for a in self.contracted_bid]
        )
        return
    
    def cancel_order(self, alpha=0.7):
        for i in range(len(self.limit_ask)):
            self.limit_ask[i][0] *= alpha
        self.limit_ask = [
            [int(a[0]), float(a[1])] for a in self.limit_ask
            if a[0] >= 1.
        ]
        for j in range(len(self.limit_bid)):
            self.limit_bid[j][0] *= alpha
        self.limit_bid = [
            [int(a[0]), float(a[1])] for a in self.limit_bid
            if a[0] >= 1.
        ]
        return

    
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
board = PriceBoard(99.0, 101.0, info=False)
asks = []
bids = []
d = []
for i in range(100):
    volumes = np.random.randint(1, 9, size=2) * 100
    board.order_ask(int(volumes[0] * 1.5))
    board.order_ask(volumes[0], board.ask - 0.5)
    board.order_bid(int(volumes[1] * 1.5))
    board.order_bid(volumes[1], board.bid + 0.5)
    board.contract()
    asks.append(board.ask)
    bids.append(board.bid)
    cont_ask_vol = np.sum([a[0] for a in board.contracted_ask])
    cont_bid_vol = np.sum([b[0] for b in board.contracted_bid])
    d.append(cont_ask_vol - cont_bid_vol)
    board.clear_market_order()
    board.cancel_order()
    
plt.subplot(121)
plt.plot(asks, color="r", label="ask")
plt.plot(bids, color="b", label="bid")
plt.legend()
plt.twinx()
plt.plot(d, color="gray", alpha=0.5)
plt.axhline(0.0, color="k")

plt.subplot(122)
lask = np.array(board.limit_ask)
lbid = np.array(board.limit_bid)
plt.barh(lask[0], lask[1], color='r', alpha=0.6)
plt.barh(lbid[0], lbid[1], color='b', alpha=0.6)

plt.tight_layout()
plt.show()

Using matplotlib backend: Qt5Agg
